In [1]:
from scipy import spatial
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import random
import time
import math
tokenizer = RegexpTokenizer(r'\w+')


#1: Create a new training set containing three equally large parts: a) all pairs of item-item or def-items from same variable (1s). b) a downsampled set of item-item or def-item pairs from the same source(id) (0s). c) a downsampled set of item-item or def-item pairs from different sources.
#2: create a test set of all item-item or def-item pairs from same poolid (1s) and a downsampled set from different poolids (0s).
#3: Create a model using the training set from #1.
#4: Predict based on the test set from #2 in DR.
#5 Compute confusion matrix + F1-scores based on results from #4.
#6: Once you are done with #1-#5, I will provide you with a new set of items and definitions that have been cleaned up more, and you'll run your code again with these items/definitions (steps #1-#5).
#7 Conclude whether the cleaned-up data helped or not (pray for not).

## Now with our data

In [2]:
ItemsDF = pd.read_excel('Items.xlsx') #header=None, skiprows=1)
#df.head(10)
#dfd = df.head(10)
ItemsDF.head(1)

,SourceId,VariableId,ItemId,ConstructName,Text,PoolID,PoolName
0,68,14,1,Knowledge transfer,During this (Purchasing) module implementation...,713.0,Perceived learning


## Create a new column in your existing dataframe

## Import the GoogleWordVec

In [3]:
import pandas as pd
import csv

import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [6]:
word_vectors = model.wv

/Users/megoconnell/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


## Create an overlapping dictionary

In [4]:
import re
all_words = [word for sentence in ItemsDF['Text'] for word in re.split('\W', sentence)]
unique = (set(all_words))

In [7]:
a = unique
c = []
for word in a:
    if word in word_vectors.vocab:
        c.append(word)


In [8]:
dict1 = {}
for word in c:
    vword = model[word]
    vector = {word : vword }
    dict1.update(vector)

In [9]:
all_words1 = [word.lower() for sentence in ItemsDF['Text'] for word in re.split('\W', sentence)]
lower = (set(all_words))

In [10]:
Embeddings1 = dict1

In [11]:
stop_words = set(stopwords.words('english'))

## Functions!

In [12]:
def calculateCosine1(W1,W2):
    #resultww = []
    result = 1 - spatial.distance.cosine(Embeddings1[W1], Embeddings1[W2])
    #if W1.lower() in weightdict:
        
    result = round(result, 2)
    return(result)

In [13]:
def calculateSentenceSim1(S1,S2):
    pairs = []
    for index1,word1 in enumerate(S1):
        for index2,word2 in enumerate(S2):
            pairs.append([index1+1, word1,index2+1,word2,calculateCosine1(word1,word2)])
    pairs = sorted(pairs, key=lambda x:x[4], reverse=True)
    
    # Now create the similarity vector
    tempPairsVector = []
    for index, row in enumerate(pairs):


        foundOne=False
        for tempRow in tempPairsVector:

            if tempRow[0] == row[0] or tempRow[2] == row[2]:
                foundOne=True
        if foundOne==False:
            tempPairsVector.append(row)

#Now create the vector of the up to 20 highest cosines
    sentenceSimVector = [] #[None] * 20
    for index,row in enumerate(tempPairsVector):
        sentenceSimVector.append(row[4])
    
#Pad the similarity vector with nulls    
    length = 20 - len(sentenceSimVector)
    for x in range(0, length):
        sentenceSimVector.append(None)            
    return(sentenceSimVector)

## Now with all the data

In [19]:
df_test = dfd.head(5)


## Set up the data

## Make a dictionary from the DataFrame

In [20]:
len(word_vectors.vocab.keys())

3000000

In [21]:
ItemsDF.head(1)

,SourceId,VariableId,ItemId,ConstructName,Text,PoolID,PoolName
0,68,14,1,Knowledge transfer,During this (Purchasing) module implementation...,713.0,Perceived learning


In [14]:
ItemsDF2 = ItemsDF[['ItemId','VariableId','Text','SourceId','PoolID']]

In [15]:
ItemsDF2.head(1)

,ItemId,VariableId,Text,SourceId,PoolID
0,1,14,During this (Purchasing) module implementation...,68,713.0


In [21]:
df_test = ItemsDF2.head(20)

In [71]:
import re
new = {}
itemslist = []
dictionary = {x[0]: { x[1] : {x[2]:x[3:]} } for x in ItemsDF2.itertuples(index=False)}
for key, item in dictionary.items():
   # print(key, item)
    for key2, item2 in item.items():
        for key3, item3 in item2.items():
            #print(item3)
            itemslist = (list(item2))
        #print(itemslist)
        #print(itemslist1)
            words = ([word for sentence in itemslist for word in tokenizer.tokenize(sentence)])
        #words = ([word for sentence in itemslist for word in tokenizer.tokenize(sentence)])
        #print(words)
        #print(words)
            filtered_sentence = [w for w in words if not w in stop_words]
            for word in filtered_sentence:
                #filtered_sentence = [w for w in words if not w in stop_words]
                if word not in Embeddings1.keys():
                    #print(word,words)
                    filtered_sentence.remove(word)
            for word in filtered_sentence:
                if word not in Embeddings1.keys():
                    #print(word,words)
                    filtered_sentence.remove(word)
            new[key] =  [key2,item3[0],item3[1], filtered_sentence]

In [67]:
#import re
#new = {}
#itemslist = []
#dictionary = {x[0]: { x[1] : x[2:] } for x in ItemsDF2.itertuples(index=False)}
#for key, item in dictionary.items():
    #print(key, item)
    #for key2, item2 in item.items():
        #itemslist.append(item2[0])
        #itemslist = (list(item2))
        #print(itemslist)
        #itemslist1 = list(itemslist[0])
        #print(itemslist1)
        #words = ([word for sentence in itemslist1 for word in tokenizer.tokenize(sentence)])
        #words = ([word for sentence in itemslist for word in tokenizer.tokenize(sentence)])
        #print(words)
        #print(words)
        #filtered_sentence = [w for w in words if not w in stop_words]
        #for word in filtered_sentence:
            #filtered_sentence = [w for w in words if not w in stop_words]
            #if word not in Embeddings1.keys():
                #print(word,words)
                #filtered_sentence.remove(word)
                #print(word,words)
                #filtered_sentence = [w for w in words if not w in stop_words]
                #print(filtered_sentence)
        #new[key] =  [key2,item2[1],item2[2] , filtered_sentence]

In [80]:
ItemsDF2.head(1)

,ItemId,VariableId,Text,SourceId,PoolID
0,1,14,During this (Purchasing) module implementation...,68,713.0


In [68]:
new

{1: [14,
  68,
  713.0,
  ['During',
   'Purchasing',
   'module',
   'implementation',
   'project',
   'interactions',
   'Consultant',
   'William',
   'increased',
   'understanding',
   'module',
   'integrates',
   'modules',
   'systems']],
 2: [14,
  68,
  713.0,
  ['During',
   'Purchasing',
   'module',
   'implementation',
   'project',
   'interactions',
   'Consultant',
   'William',
   'increased',
   'ability',
   'ask',
   'penetrating',
   'questions',
   'module']],
 3: [14,
  68,
  713.0,
  ['During',
   'Purchasing',
   'module',
   'implementation',
   'project',
   'interactions',
   'Consultant',
   'William',
   'improved',
   'knowledge',
   'module']],
 4: [14,
  68,
  713.0,
  ['During',
   'Purchasing',
   'module',
   'implementation',
   'project',
   'interactions',
   'Consultant',
   'William',
   'increased',
   'knowledge',
   'module',
   'training',
   'documents',
   'end',
   'users']],
 5: [14,
  68,
  713.0,
  ['During',
   'Purchasing',
   'mod

In [86]:
table = []
start = time.time()
random.seed(1)
for key, value in new.items():
    for key2, values2 in new.items():
        #print(value[3])
        if key < key2:
            if value[0]==values2[0]:
                table.append([key, key2,value[0]* values2[0],1, calculateSentenceSim1(value[3],values2[3])])
            elif value[1]==values2[1]:
                foundnum=random.randint(1,1584)
                if foundnum == 1:
                        #print(value[1],values2[1])
                    table.append([key, key2,value[0]* values2[0],0, calculateSentenceSim1(value[3],values2[3])])
            elif value[1]!=values2[1]:
                foundnum=random.randint(1,1584)
                if foundnum == 1:
                    table.append([key, key2,value[0]* values2[0],0, calculateSentenceSim1(value[3],values2[3])])
table

[[1,
  2,
  196,
  1,
  [1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.26,
   0.14,
   0.09,
   0.03,
   None,
   None,
   None,
   None,
   None,
   None]],
 [1,
  3,
  196,
  1,
  [1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.63,
   0.59,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None]],
 [1,
  4,
  196,
  1,
  [1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.63,
   0.29,
   0.15,
   0.04,
   None,
   None,
   None,
   None,
   None,
   None]],
 [1,
  5,
  196,
  1,
  [1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.63,
   0.54,
   0.5,
   0.17,
   0.16,
   None,
   None,
   None,
   None,
   None,
   None]],
 [1,
  6,
  196,
  1,
  [1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.59,
   0.32,
   0.3,
   0.18,
   0.16,
   None,
   None,
   None,
   None,
   None,
   None]],
 [1,
  28010,
  385350,
  0,


In [87]:
len(table)

151080

### 50212 trues

In [88]:
list1 = []
list1.append([])
list1 = []
list1.append([])
z = 0
for x in table:
    list2 = []
    for y in x[4]:
        
        #print(y)
        #print(x[0],x[1],x[2], x[3],y)
        list2.append(y)
        #for m in list2:
            #list3.append(m)
            #list3[z].extend(m)
    list1[z].append(x[0])
    list1[z].append(x[1])
    list1[z].append(x[2])
    list1[z].append(x[3])
    list1[z].extend(list2)
    list1[z]
    z += 1
    list1.append([])
list1[0]
#list3

[1,
 2,
 196,
 1,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.26,
 0.14,
 0.09,
 0.03,
 None,
 None,
 None,
 None,
 None,
 None]

In [89]:
tableDF2 = pd.DataFrame(list1)


In [90]:
tableDF2 = tableDF2.rename(columns = {0:'ItemId1', 1:'ItemId2',2:'VId',3:'Target'})
tableDF2.head(7)

,ItemId1,ItemId2,VId,Target,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,66
0,1.0,2.0,196.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,3.0,196.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,4.0,196.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,5.0,196.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,6.0,196.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,28010.0,385350.0,0.0,1.00,0.51,0.36,0.35,0.33,0.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,28109.0,387576.0,0.0,0.63,0.58,0.37,0.28,0.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Downsample

In [91]:
GoogleNormal = tableDF2.iloc[:,0:24]

In [92]:
GoogleNormal.to_csv('GoogleNormalPooling.csv')